## 过拟合与欠拟合


### 1. 欠拟合 (Underfitting)

- **模型形式**
  $\hat{y}=w_1x + b$

- **现象**
  - 直线难以捕捉价格–面积之间的真实曲线关系
  - 训练误差依然很高

- **结论**
  - **高偏差 (high bias)**
  - “学得太少”，模型过于简单 → 训练集和测试集都表现差


### 2. 适度拟合 (Just Right)

- **模型形式**
  $\hat{y}=w_1x + w_2x^2 + b$

- **现象**
  - 曲线能较好穿过大多数样本
  - 训练误差低，验证误差也低

- **结论**
  - 较好地平衡了 **偏差** 和 **方差**
  - 具有良好的 **泛化能力(generalization)**


### 3. 过拟合 (Overfitting)

- **模型形式**
  $\hat{y}=w_1x + w_2x^2 + w_3x^3 + w_4x^4 + b$

- **现象**
  - 高阶多项式“蜿蜒”穿过所有训练点
  - 训练误差几乎为零，但在新样本（粉色圆点）处预测极差

- **结论**
  - **高方差 (high variance)**
  - “学得太多”，对训练集中的噪声也一并学习 → 测试集表现差



### 何时会出现？

| 场景 | 可能导致欠拟合 | 可能导致过拟合 |
|------|---------------|---------------|
| **模型容量** | 模型太简单、特征不足 | 模型过于复杂、高维特征堆叠 |
| **数据量** | 样本极少，难以学到规律 | 数据量不足以支撑复杂模型 |



###  分类问题里的过拟合和欠拟合
![分类的过拟合和欠拟合](./images/监督学习-分类4.png)

### 解决过拟合问题

1. 收集更多数据

![收集更多数据](./images/监督学习-回归11.png)

2. 选择要加入训练的特征

假设你手里有一大堆与房价相关的特征——面积、卧室数量、楼层数、房龄、社区平均收入、到最近咖啡店的距离……总共也许有 100 个特征，而你的训练样本又不够多。这样一来，学习算法也很容易对训练集过拟合。
解决方法之一是 挑选一部分最有用的特征。比方说，你觉得面积、卧室数量和房龄最关键，那就只用这几个；往往这样一来，模型的过拟合就会明显减轻。这个过程常被称作 特征选择 (feature selection)。你可以靠直觉和业务经验来决定哪些特征更重要，从而挑出一组“最佳”特征。
不过，特征选择的缺点是：只用子集就意味着丢弃了其他信息。也许这 100 个特征里，每一个对预测房价都有价值；一旦你扔掉其中一些，算法就可能失去部分有用信息。


3. 正则化

- 正则化的作用，是鼓励学习算法把参数数值“压缩”得更小，而不一定非得把参数直接设为 0。即使你拟合了这样的高阶多项式，只要能让算法把参数 $w_1、w_2、w_3、w_4$ 的数值减小，最终得到的曲线往往能更好地拟合训练数据。换句话说，正则化允许你保留所有特征，但阻止某些特征产生过大的影响──而过大的影响正是导致过拟合的常见原因。

![正则化](./images/监督学习-回归12.png)

### 正则化实现
#### 正则化直觉理解


- 假设我们用多项式来预测：

$$
\hat{y}= w_1\,x \;+\; w_2\,x^{2} \;+\; w_3\,x^{3} \;+\; w_4\,x^{4} \;+\; b
$$

* **低阶项**（$w_1x,\,w_2x^2$）往往描述了主要趋势。
* **高阶项**（$w_3x^3,\,w_4x^4$）容易让模型在训练集上“拧来拧去”，对噪声也拟合得很好 → 过拟合。


- 思路：让高阶权重变“极小”

如果我们能把 $w_3,w_4$ 压到几乎为0，模型就退化成

$$
\hat{y}\;\approx\; w_1\,x + w_2\,x^{2} + b
$$

仍保留了足够灵活度，但不会在训练点之间剧烈振荡。**正则化**就是专门做这件事：在损失函数里加一项“惩罚”一些权重。



- 以 **L2正则（权重衰减）** 为例

$$
\underset{\mathbf w,b}{\min}\;\;
\frac{1}{2m}\sum_{i=1}^{m}\bigl(f_{\mathbf w,b}(\mathbf x^{(i)})-y^{(i)}\bigr)^2
\;+\; \lambda\bigl(w_3^{\,2}+w_4^{\,2}\bigr)
$$

* 第一部分仍是 **均方误差**。
* 第二部分 $\lambda w_k^{2}$ 就像“弹簧”——权重越大，弹簧拉力（惩罚）越大。
* 只要 $\lambda$ 选得合适，优化算法自然会把 $w_3,w_4$ 压到接近0，而不影响主要的 $w_1,w_2$。

> 比如，$\lambda\approx 1000$ 时，优化过程会自动缩写w3，w4。
> $w_3^2\approx0.001,\;w_4^2\approx0.002$，说明它们被大幅缩小。



- 为什么这招能防过拟合？

| 直觉            | 解释                                      |
| ------------- | --------------------------------------- |
| **“模型更平滑”**   | 大权重意味着输出对输入微小波动极敏感；缩权重 → 曲线变平滑。         |
| **“噪声信号被抑制”** | 噪声往往需要复杂、高振幅的函数来拟合；当权重受限，模型只能关注真正的全局趋势。 |
| **“容量控制”**    | 不删除特征，而是限制它们的影响力 ⇒ 在保留信息的同时降低方差。        |




#### 实现方式
- 更小的权重（w1到wn），更简单的模型 -> 更不会过拟合数据

- 带L2正则化的线性回归成本函数

$$
J(\,\vec w,\,b\,)
\;=\;
\frac{1}{2m}\,
\underbrace{\sum_{i=1}^{m}
        \bigl(f_{\vec w,b}(\vec x^{(i)})-y^{(i)}\bigr)^{2}}_{\text{均方误差 (MSE)}}
\;+\;
\frac{\lambda}{2m}\,
\underbrace{\sum_{j=1}^{n} w_j^{2}}_{\text{正则化项}}
\;+\;
\boxed{\;
\frac{\lambda}{2m}\,b^{2}\;}_{\substack{\text{可选：}\\\text{是否对 } b \text{ 也正则化}}}
$$


- 各部分含义

| 符号                                                      | 含义                             |
| ------------------------------------------------------- | ------------------------------ |
| $m$                                                     | 训练样本数                          |
| $n$                                                     | 特征数量                           |
| $f_{\vec w,b}(\vec x)$                                  | 预测函数（线性或多项式）                   |
| $\lambda>0$                                             | **正则化系数**（“lambda”），决定惩罚强度     |
| $\displaystyle\frac{\lambda}{2m}\sum_{j=1}^{n} w_j^{2}$ | **L2 正则化项**：约束权重 $\vec w$ 不宜过大 |
| $\displaystyle\frac{\lambda}{2m} b^{2}$                 | 可选择是否一并惩罚偏置 $b$                |


直觉说明：

* **第一项**（MSE）——让模型尽量拟合训练数据。
* **第二/三项**（正则化）——给权重“上沙袋”，权值越大惩罚越重：

  * 抑制高阶或噪声相关特征的影响；
  * 减少过拟合，使模型更平滑、泛化更好。

> 实践中常把偏置 $b$ 排除在正则化之外（即不含最后那一项），以免整体平移也受惩罚；但两种写法理论上都成立，视具体实现而定。


- 除了学习率外，要选择合适的正则化参数 lambda
- lambda/2m后，更容易找到合适的lambda
- lambda的值在拟合数据 和 降低权重之间 找到平衡

#### 线性回归正则化

1. 成本函数（含正则项）

$$
\boxed{%
J(\vec w,b)=
\frac{1}{2m}\sum_{i=1}^{m}\bigl(f_{\vec w,b}(\vec x^{(i)})-y^{(i)}\bigr)^2
\;+\;
\frac{\lambda}{2m}\sum_{j=1}^{n} w_j^{2}}
$$

* 前半部分：**均方误差 (MSE)**
* 后半部分：**正则化项**

  * $\lambda>0$ 为正则化系数
  * 约束权重 $\vec w$ 不宜过大
  * **不** 对偏置 $b$ 正则化（如需，也可同样加一项 $\lambda b^{2}/(2m)$）



2. 各参数的梯度

| 参数    | 梯度表达式                                                                                                                                                            | 由来                    |
| ----- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------- |
| $w_j$ | $\displaystyle \frac{\partial J}{\partial w_j}= \frac{1}{m}\sum_{i=1}^{m}\bigl(f_{\vec w,b}(\vec x^{(i)})-y^{(i)}\bigr)\,x^{(i)}_j \;+\; \frac{\lambda}{m}\,w_j$ | 误差梯度 $+$ 正则梯度         |
| $b$   | $\displaystyle \frac{\partial J}{\partial b}= \frac{1}{m}\sum_{i=1}^{m}\bigl(f_{\vec w,b}(\vec x^{(i)})-y^{(i)}\bigr)$                                           | 仅误差梯度；若不正则化 $b$ 则无额外项 |



3. 批量梯度下降更新规则（同时更新）

$$
\begin{cases}
w_j \;\leftarrow\; w_j \;-\; \alpha\,
        \bigl[\;\frac{1}{m}\!\sum_{i}(f-y)x_j \;+\; \frac{\lambda}{m}w_j\bigr],
        & j=1,\dots,n \\[10pt]
b   \;\leftarrow\; b   \;-\; \alpha\,
        \bigl[\;\frac{1}{m}\!\sum_{i}(f-y)\bigr]
\end{cases}
$$

- 正则化后，wj每次更新发生了什么？

1. 带L2正则化时，单个权重 $w_j$ 的梯度下降更新公式

$$
\boxed{%
w_j
\;=\;
w_j
\;-\;
\underbrace{\alpha \frac{\lambda}{m} w_j}_{\text{正则化―“收缩”}}
\;-\;
\underbrace{\alpha \frac{1}{m}\sum_{i=1}^{m}
          \bigl(f_{\vec w,b}(\vec x^{(i)})-y^{(i)}\bigr)\,x^{(i)}_j}_{\text{常规误差梯度}}
}
$$



2. 公式调整

$$
w_j
\;=\;
\color{orange}{w_j}
\Bigl(1 - \alpha \tfrac{\lambda}{m}\Bigr)
\;\;-\;\;
\alpha \frac{1}{m}\sum_{i}(f-y)x_j
$$

* 前半部分：$\bigl(1 - \alpha\lambda/m\bigr)$ 类似给 $w_j$ 乘上一个略小于1的系数 → **把权重整体“压缩”一小点**。
* 后半部分：正常的梯度更新，用于减小预测误差。



3. 数值示例

设

* 学习率 $\alpha = 0.01$
* 正则化系数 $\lambda = 1$
* 样本数 $m = 50$

则

$$
\alpha \frac{\lambda}{m}
= 0.01 \times \frac{1}{50}
= 0.0002
$$

收缩系数

$$
1 - 0.0002 = 0.9998
$$

即 **每一步**，仅正则化这一项就会把现有 $w_j$ 乘以 0.9998──权重被轻微缩小，而不会骤降。误差梯度项仍照常调整方向，使模型既学习数据规律，又缩小权重避免过拟合。


#### 逻辑回归正则化

1. 成本函数（Cost)

$$
\boxed{%
J(\vec w,b)=
-\frac1m\sum_{i=1}^{m}
\bigl[
y^{(i)}\log f_{\vec w,b}(\vec x^{(i)})
+\bigl(1-y^{(i)}\bigr)\log\!\bigl(1-f_{\vec w,b}(\vec x^{(i)})\bigr)
\bigr]
\;+\;
\frac{\lambda}{2m}\sum_{j=1}^{n}w_j^{2}}
$$

* **左侧**：逻辑回归的损失函数
* **右侧**：L2正则化项

  * $\lambda>0$：正则化系数，越大“惩罚”越强


2. 梯度

| 参数    | 梯度表达式                                                                                                                             |
| ----- | --------------------------------------------------------------------------------------------------------------------------------- |
| $w_j$ | $\displaystyle \frac{\partial J}{\partial w_j}= \frac1m\sum_{i=1}^{m}\bigl(f^{(i)}-y^{(i)}\bigr)x_j^{(i)} + \frac{\lambda}{m}w_j$ |
| $b$   | $\displaystyle \frac{\partial J}{\partial b}= \frac1m\sum_{i=1}^{m}\bigl(f^{(i)}-y^{(i)}\bigr)$                                   |

*其中 $f^{(i)}=f_{\vec w,b}(\vec x^{(i)})=\sigma(\vec w^{\!\top}\vec x^{(i)}+b)$*


3. 批量梯度下降（同时更新）

$$
\begin{cases}
w_j \;\leftarrow\; w_j-\alpha\Bigl[\tfrac1m\sum_{i}(f^{(i)}-y^{(i)})x^{(i)}_j + \tfrac{\lambda}{m}w_j\Bigr] \\[10pt]
b   \;\leftarrow\; b  -\alpha\Bigl[\tfrac1m\sum_{i}(f^{(i)}-y^{(i)})\Bigr]
\end{cases}
$$

> **收缩视角**
> $w_j$ 其实每步都被乘上 $(1-\alpha\lambda/m)$ 再加正常梯度修正——
> 相当于“先整体缩小，再学习误差”。
